In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split

In [4]:
df=pd.read_csv('turnover.csv',encoding='ISO-8859-1')

In [5]:
df.head()

,stag,event,gender,age,industry,profession,traffic,coach,head_gender,greywage,way,extraversion,independ,selfcontrol,anxiety,novator
0,7.030801,1,m,35.0,Banks,HR,rabrecNErab,no,f,white,bus,6.2,4.1,5.7,7.1,8.3
1,22.965092,1,m,33.0,Banks,HR,empjs,no,m,white,bus,6.2,4.1,5.7,7.1,8.3
2,15.934292,1,f,35.0,PowerGeneration,HR,rabrecNErab,no,m,white,bus,6.2,6.2,2.6,4.8,8.3
3,15.934292,1,f,35.0,PowerGeneration,HR,rabrecNErab,no,m,white,bus,5.4,7.6,4.9,2.5,6.7
4,8.410678,1,m,32.0,Retail,Commercial,youjs,yes,f,white,bus,3.0,4.1,8.0,7.1,3.7


Check for missing values and column data types


In [9]:
df.isnull().sum()

,0
stag,0
event,0
gender,0
age,0
industry,0
profession,0
traffic,0
coach,0
head_gender,0
greywage,0


In [10]:
df.dtypes

,0
stag,float64
event,int64
gender,object
age,float64
industry,object
profession,object
traffic,object
coach,object
head_gender,object
greywage,object


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1129 entries, 0 to 1128
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   stag          1129 non-null   float64
 1   event         1129 non-null   int64  
 2   gender        1129 non-null   object 
 3   age           1129 non-null   float64
 4   industry      1129 non-null   object 
 5   profession    1129 non-null   object 
 6   traffic       1129 non-null   object 
 7   coach         1129 non-null   object 
 8   head_gender   1129 non-null   object 
 9   greywage      1129 non-null   object 
 10  way           1129 non-null   object 
 11  extraversion  1129 non-null   float64
 12  independ      1129 non-null   float64
 13  selfcontrol   1129 non-null   float64
 14  anxiety       1129 non-null   float64
 15  novator       1129 non-null   float64
dtypes: float64(7), int64(1), object(8)
memory usage: 141.2+ KB


In [13]:
df.describe()

,stag,event,age,extraversion,independ,selfcontrol,anxiety,novator
count,1129.000000,1129.000000,1129.000000,1129.000000,1129.000000,1129.000000,1129.000000,1129.000000
mean,36.627526,0.505757,31.066965,5.592383,5.478034,5.597254,5.665633,5.879628
std,34.096597,0.500188,6.996147,1.851637,1.703312,1.980101,1.709176,1.904016
min,0.394251,0.000000,18.000000,1.000000,1.000000,1.000000,1.700000,1.000000
25%,11.728953,0.000000,26.000000,4.600000,4.100000,4.100000,4.800000,4.400000
50%,24.344969,1.000000,30.000000,5.400000,5.500000,5.700000,5.600000,6.000000
75%,51.318275,1.000000,36.000000,7.000000,6.900000,7.200000,7.100000,7.500000
max,179.449692,1.000000,58.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [14]:
from sklearn.preprocessing import LabelEncoder
print('Categorical columns: ')
for col in df.columns:
    if df[col].dtype == 'object':
        values = df[col].value_counts()
        values = dict(values)

        print(str(col))
        label = LabelEncoder()
        label = label.fit(df[col])
        df[col] = label.transform(df[col].astype(str))

        new_values = df[col].value_counts()
        new_values = dict(new_values)

        value_dict = {}
        i=0
        for key in values:
            value_dict[key] = list(new_values)[i]
            i+= 1
        print(value_dict)

Categorical columns: 
gender
{'f': 0, 'm': 1}
industry
{'Retail': 10, 'manufacture': 14, 'IT': 5, 'Banks': 2, 'etc': 13, 'Consult': 4, 'State': 11, 'Building': 3, 'PowerGeneration': 8, 'transport': 15, 'Telecom': 12, 'Mining': 6, 'Pharma': 7, 'Agriculture': 1, 'RealEstate': 9, ' HoReCa': 0}
profession
{'HR': 6, 'IT': 7, 'Sales': 11, 'etc': 13, 'Marketing': 9, 'BusinessDevelopment': 1, 'Consult': 3, 'Commercial': 2, 'manage': 14, 'Finanñe': 5, 'Engineer': 4, 'Teaching': 12, 'Accounting': 0, 'Law': 8, 'PR': 10}
traffic
{'youjs': 7, 'empjs': 2, 'rabrecNErab': 4, 'friends': 3, 'referal': 6, 'KA': 0, 'recNErab': 5, 'advert': 1}
coach
{'no': 1, 'my head': 0, 'yes': 2}
head_gender
{'m': 1, 'f': 0}
greywage
{'white': 1, 'grey': 0}
way
{'bus': 0, 'car': 1, 'foot': 2}


EDA

In [16]:
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier, ExtraTreesClassifier
df_corr = df.corr()
fig = px.imshow(df_corr)
fig.show()

In [17]:
fig = px.pie(df, "event", color='event', hole=.3)
fig.show()

In [18]:
fig = px.histogram(df, x="stag", color='event', marginal='box', barmode='group')
fig.show()

In [19]:
fig = px.histogram(df, x="age", color='event', marginal='box', barmode='group')
fig.show()

In [20]:
fig = px.histogram(df, x="gender", color='event', barmode='group')
fig.show()

In [21]:
X = df.drop(columns=['event'])
y = df['event']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2)

In [23]:
models = {}
def train_validate_predict(classifiers, x_train, y_train, x_test, y_test, index):
    model = classifiers
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    r2 = accuracy_score(y_test, y_pred)
    models[index] = r2

In [24]:
model_names = ['DecisionTreeClassifier','LogisticRegression']
model_list = [DecisionTreeClassifier,LogisticRegression]

index = 0
for classifiers in model_list:
    train_validate_predict(classifiers(), X_train, y_train, X_test, y_test, model_names[index])
    index+=1

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [25]:
models

{'DecisionTreeClassifier': 0.6592920353982301,
 'LogisticRegression': 0.5221238938053098}